In [21]:
from agno.agent import Agent
from agno.workflow import Workflow,StepInput,StepOutput,Parallel,Step
from pydantic import BaseModel, Field
from agno.models.mistral import MistralChat
from agno.tools.mcp import MCPTools
from agno.tools.wikipedia import WikipediaTools
from agno.tools.local_file_system import LocalFileSystemTools
from agno.tools.newspaper4k import Newspaper4kTools
from agno.tools.hackernews import HackerNewsTools
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.youtube import YouTubeTools
from agno.tools.arxiv import ArxivTools
from mcp import StdioServerParameters
from agno.team import Team
from agno.models.groq import Groq
from agno.models.openrouter import OpenRouter
from agno.tools.reddit import RedditTools
from pathlib import Path
from textwrap import dedent
from dotenv import load_dotenv
import json
import os

load_dotenv()

True

In [3]:
if os.getenv('GROQ_API_KEY'):
    model0=Groq(api_key=os.getenv('GROQ_API_KEY'))
else:
    print('NOT THEERE')

In [4]:
model=MistralChat()

In [5]:
research_path=Path('./arxiv_paper/')

research_path.mkdir(exist_ok=True)

medium_file=Path('./medium_article/')
medium_file.mkdir(exist_ok=True)


In [6]:

wiki=WikipediaTools(all=True)

hn=HackerNewsTools(all=True)

newspaper=Newspaper4kTools(include_summary=True)

ddgs=DuckDuckGoTools(fixed_max_results=5,cache_results=True)


In [7]:
youtube=YouTubeTools(all=True)


In [ ]:

arxiv_agent=Agent(name='arxiv_agent',
                  tool_call_limit=3,
                   instructions=["You are a research assistant that gathers research papers from Arxiv",
                  "Use the available tools to search for research papers, authors and topics as per user's request",
                  "Summarize your findings clearly and concisely and you can download the research paper using arxiv tool "],
                  role='arxic_researcher',
                  model=model,
                  tools=[ArxivTools(all=True,download_dir=research_path)],
                  add_datetime_to_context=True
                  )

# Create Wikipedia agent
wikipedia_agent = Agent(
    name='wikipedia_agent',
    tool_call_limit=3,
    instructions=[
        "You are a research assistant that gathers information from Wikipedia",
        "Use the available tools to search for articles, topics, and summaries as per user's request",
        "Summarize your findings clearly and concisely"
    ],
    role='wikipedia_researcher',
    model=model,
    tools=[WikipediaTools(all=True)],
    add_datetime_to_context=True
)

# Create HackerNews agent
hackernews_agent = Agent(
    name='hackernews_agent',
    tool_call_limit=3,
    instructions=[
        "You are a research assistant that gathers information from HackerNews",
        "Use the available tools to search for stories, comments, and topics as per user's request",
        "Summarize your findings clearly and concisely"
    ],
    role='hackernews_researcher',
    model=model,
    tools=[HackerNewsTools(all=True)],
    add_datetime_to_context=True
)

# Create Newspaper agent
newspaper_agent = Agent(
    name='newspaper_agent',
    tool_call_limit=3,
    instructions=[
        "You are a research assistant that gathers information from news articles",
        "Use the available tools to search for articles, topics, and summaries as per user's request",
        "Summarize your findings clearly and concisely"
    ],
    role='newspaper_researcher',
    model=model,
    tools=[Newspaper4kTools(include_summary=True)
],
    add_datetime_to_context=True
)

# Create DuckDuckGo agent
duckduckgo_agent = Agent(
    name='duckduckgo_agent',
    tool_call_limit=3,
    instructions=[
        "You are a research assistant that gathers information using DuckDuckGo search engine",
        "Use the available tools to search for web pages, topics, and summaries as per user's request",
        "Summarize your findings clearly and concisely"
    ],
    role='duckduckgo_researcher',
    model=model,
    tools=[DuckDuckGoTools(fixed_max_results=5,cache_results=True)],
    add_datetime_to_context=True
)

research_team=Team(
    members=[arxiv_agent,wikipedia_agent,hackernews_agent,newspaper_agent,duckduckgo_agent],
    name='research_team',
    role='leader',
    model=model,
    instructions=['you are the leader of the research team and your goal is to gather information from different sources and summarize it in a clear and concise manner make sure you use the filesystem tool to save the information gathered in the directory and use you sub agents like duck duck go for duck duck go search, newspaper agent to gather news articles and hackernews agent to get hacker news information and wikipedia agent to get wikipedia information and arxiv agent to get arxiv information and write the article in simple plain english wih no jargons and cite the sources also with the metrics if used  and dont give output beyond the scope of the query you get ' ],
    add_datetime_to_context=True,
    add_history_to_context=True

)

yt_agent=Agent(
    model=model0,
    name='youtube_agent',
    role='youtube_researcher',
    instructions=['you are a youtube agent that searches for videos and gives the transcript of the video'],
    tools=[YouTubeTools(all=True)
],
    add_datetime_to_context=True
)


yt_team=Team(
    members=[yt_agent],
    name='youtube_team',
    role='leader',
    model=model0,
    instructions=['you are the leader of the youtube team and your goal is to gather information from youtube and summarize it in a clear and concise manner'],
    add_datetime_to_context=True,
)

In [16]:
#build the workflow

step1=Step(
    name='step1',
    team=research_team,
    description='a team dedicated for doing research over the web'
)

step2=Step(
    name='step2',
    team=yt_team,
    description='a team dedicated for doing research over youtube'
)

In [17]:
paralell=Parallel(
    step1,step2,
    name='parallel',
    description='perform search on the topic given comprehensive advanced research'
)

In [18]:
agent=Agent(
    model=model0,
    role='report generator',
    name='report_generator',
    instructions=['you are a report generator that generates a report based on the information gathered by the research team and the youtube team using the tools you have and save the report to the given directory in ur local file system make sure it is in simple words and engaging with no jargons and cite the sources also with the metrics if used  and dont give output beyond the scope of the query you get and make sure you dont hallucinate'],
     tools=[LocalFileSystemTools(target_directory=medium_file,default_extension='md')],
     add_datetime_to_context=True,
     id='report_generator'
    
)

In [19]:
wf=Workflow(
    id='workflow medium',
    steps=[paralell,agent],
    description='a workflow that performs comprehensive advanced research on a given topic and generates a report based on the information gathered',
    name='workflow medium',
    
)

In [20]:
wf.run('do a deep research about the topic of agnetic ai and make a blog on it with a clear and concise summary of the findings and cite the sources also with the metrics if used  and dont give output beyond the scope of the query you get and make sure you dont hallucinate')

WARNING  add_history_to_context is True, but no database has been assigned to the team. History will not be added  
         to the context.

ERROR    Error calling Groq API: Error code: 400 - {'error': {'message': 'tool call validation failed: attempted to
         call tool \'get_youtube_video_captions,{"url":                                                            
         "https://www.youtube.com/watch?v=example_magnetic_ai_video"}\' which was not in request.tools', 'type':   
         'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation':                                  
         '<function=get_youtube_video_captions,{"url":                                                             
         "https://www.youtube.com/watch?v=example_magnetic_ai_video"}></function>\n'}}

ERROR    Error in Agent run: {"error":{"message":"tool call validation failed: attempted to call tool              
         'get_youtube_video_captions,{\"url\": \"https://www.youtube.com/watch?v=example_magnetic_ai_video\"}'     
         which was not in                                                                                          
         request.tools","type":"invalid_request_error","code":"tool_use_failed","failed_generation":"\u003cfunction
         =get_youtube_video_captions,{\"url\":                                                                     
         \"https://www.youtube.com/watch?v=example_magnetic_ai_video\"}\u003e\u003c/function\u003e\n"}}

ERROR    Error calling Groq API: Error code: 429 - {'error': {'message': 'Rate limit reached for model             
         `llama-3.3-70b-versatile` in organization `org_01jzwrgb19e85bx6gmwgh6ks9n` service tier `on_demand` on    
         tokens per day (TPD): Limit 100000, Used 99542, Requested 794. Please try again in 4m50.304s. Need more   
         tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code':
         'rate_limit_exceeded'}}

ERROR    Error in Agent run: {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in        
         organization `org_01jzwrgb19e85bx6gmwgh6ks9n` service tier `on_demand` on tokens per day (TPD): Limit     
         100000, Used 99542, Requested 794. Please try again in 4m50.304s. Need more tokens? Upgrade to Dev Tier   
         today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}

...

INFO Searching wikipedia for: Key concepts and frameworks in Agentic AI

ERROR    Error in Agent run: Response validation failed: 5 validation errors for Unmarshaller                      
         body.choices.0.message.content.nullable[union[str,list[tagged-union[ImageURLChunk,DocumentURLChunk,TextChu
         nk,ReferenceChunk,FileChunk,ThinkChunk,AudioChunk]]]].str                                                 
           Input should be a valid string [type=string_type, input_value=[{'type': 'text', 'text':...elligence*.   
         [Link](#)'}], input_type=list]                                                                            
             For further information visit https://errors.pydantic.dev/2.12/v/string_type                          
         body.choices.0.message.content.nullable[union[str,list[tagged-union[ImageURLChunk,DocumentURLChunk,TextChu
         nk,ReferenceChunk,FileChunk,ThinkChunk,AudioChunk]]]].list[tagged-union[ImageURLChunk,DocumentURLChunk,Tex
         tChunk,ReferenceChunk,FileChunk,ThinkChunk,AudioChunk]].1.reference.reference_ids.0                       
           Input should be a valid integer, unable to parse string as an integer [type=int_parsing,                
         input_value='history', input_type=str]                                                                    
             For further information visit https://errors.pydantic.dev/2.12/v/int_parsing                          
         body.choices.0.message.content.nullable[union[str,list[tagged-union[ImageURLChunk,DocumentURLChunk,TextChu
         nk,ReferenceChunk,FileChunk,ThinkChunk,AudioChunk]]]].list[tagged-union[ImageURLChunk,DocumentURLChunk,Tex
         tChunk,ReferenceChunk,FileChunk,ThinkChunk,AudioChunk]].3.reference.reference_ids.0                       
           Input should be a valid integer, unable to parse string as an integer [type=int_parsing,                
         input_value='history', input_type=str]                                                                    
             For further information visit https://errors.pydantic.dev/2.12/v/int_parsing                          
         body.choices.0.message.content.nullable[union[str,list[tagged-union[ImageURLChunk,DocumentURLChunk,TextChu
         nk,ReferenceChunk,FileChunk,ThinkChunk,AudioChunk]]]].list[tagged-union[ImageURLChunk,DocumentURLChunk,Tex
         tChunk,ReferenceChunk,FileChunk,ThinkChunk,AudioChunk]].5.reference.reference_ids.0                       
           Input should be a valid integer, unable to parse string as an integer [type=int_parsing,                
         input_value='history', input_type=str]                                                                    
             For further information visit https://errors.pydantic.dev/2.12/v/int_parsing                          
         body.choices.0.message.content.Unset                                                                      
           Input should be a valid dictionary or instance of Unset [type=model_type, input_value=[{'type': 'text', 
         'text':...elligence*. [Link](#)'}], input_type=list]                                                      
             For further information visit https://errors.pydantic.dev/2.12/v/model_type

ERROR    Error calling Groq API: Error code: 429 - {'error': {'message': 'Rate limit reached for model             
         `llama-3.3-70b-versatile` in organization `org_01jzwrgb19e85bx6gmwgh6ks9n` service tier `on_demand` on    
         tokens per day (TPD): Limit 100000, Used 99907, Requested 788. Please try again in 10m0.48s. Need more    
         tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code':
         'rate_limit_exceeded'}}

ERROR    Error in Agent run: {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in        
         organization `org_01jzwrgb19e85bx6gmwgh6ks9n` service tier `on_demand` on tokens per day (TPD): Limit     
         100000, Used 99907, Requested 788. Please try again in 10m0.48s. Need more tokens? Upgrade to Dev Tier    
         today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}

WorkflowRunOutput(input='do a deep research about the topic of agnetic ai and make a blog on it with a clear and concise summary of the findings and cite the sources also with the metrics if used  and dont give output beyond the scope of the query you get and make sure you dont hallucinate', content='{"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jzwrgb19e85bx6gmwgh6ks9n` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99907, Requested 788. Please try again in 10m0.48s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}\n', content_type='str', workflow_id='workflow medium', workflow_name='workflow medium', run_id='6cd4edb9-b568-4a2a-8f76-db2be9c0dbcf', session_id='cd27f53e-a204-44af-91cf-f7dcc640f96f', user_id=None, images=[], videos=[], audio=[], response_audio=None, step_results=[StepOutput(step_name='parallel', step_id='aa5481